In [1]:
# passivate/activate method
# https://www.salabim.org/manual/Modelling.html#a-bank-example

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(True)
        self.status.monitor(True)

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)

    def process(self):
        self.enter(waitingline)
        for ChargingStation in ChargingStations:
            if ChargingStation.ispassive():
                ChargingStation.activate()
                break  # activate at most one charging station
        self.passivate()


class ChargingStation(sim.Component):
    def setup(self):
        self.mode.monitor(True)
        self.status.monitor(True)
        self.proc = sim.Monitor(name="TST.", monitor=True, level=False, type="float")

    def process(self):
        while True:
            ws = app.now()
            while len(waitingline) == 0:
                self.set_mode("IDLE")
                self.passivate(mode="IDLE")
            self.car = waitingline.pop()
            wf = app.now()
            charging_time = srv_distr.sample()
            monEVSE_IDLE.tally(wf - ws)
            monEVSE_PROC.tally(charging_time)
            self.proc.tally(charging_time)
            self.set_mode("PROC")
            self.hold(charging_time)
            self.car.activate()


N_STATION = 1
iat_distr = sim.Exponential(60 / (48 * N_STATION))
srv_distr = sim.Exponential(60 / (50 * N_STATION))

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

#
monEVSE_IDLE = sim.Monitor(name="IDLE", monitor=True, level=False, type="float")
monEVSE_PROC = sim.Monitor(name="PROC", monitor=True, level=False, type="float")

# Create Queue and set monitor to stats_only
waitingline = sim.Queue(name="Waiting Cars", monitor=True)
waitingline.length_of_stay.monitor(value=True)
waitingline.length_of_stay.reset_monitors(stats_only=False)

# Instantiate the servers, list comprehension
ChargingStations = [ChargingStation() for _ in range(N_STATION)]


# Execute Simulation
app.run(till=50000)

# Print queueing statistics
# waitingline.length_of_stay.print_statistics()

# import matplotlib.pyplot as plt

# plt.plot(*waitingline.length.tx(), linewidth=1, color="red")
# plt.show()


monEVSE_IDLE.animate(
    x=10, y=450, width=480, height=100, horizontal_scale=5, vertical_scale=5
)

# Print server statistics
I = monEVSE_IDLE.mean()
P = monEVSE_PROC.mean()
R = (P * monEVSE_PROC.number_of_entries()) / (
    (I * monEVSE_IDLE.number_of_entries()) + (P * monEVSE_PROC.number_of_entries())
)

print(f"IDLE: {I}")
print(f"PROC: {P}")
print(f"UTIL: {R}")


def prtEVSE(evse):
    print(
        f"name: {evse.proc.name()}, seq: {evse.proc.sequence_number()}, mean: {evse.proc.mean()}"
    )


[prtEVSE(x) in ChargingStations for x in ChargingStations]

print(f"Total: {sum(x.proc for x in ChargingStations).mean()}")
ChargingStations[0].status.print_histogram(values=True)
ChargingStations[0].mode.print_histogram(values=True)

IDLE: 0.062278801977539544
PROC: 1.193105570597679
UTIL: 0.9503906505943
name: TST.1, seq: 1, mean: 1.193105570597679
Total: 1.193105570597679
Histogram of chargingstation.1.status
duration         50000    

value                     duration     %
passive                   2480.565   5.0 ***
scheduled                47519.435  95.0 ****************************************************************************

Histogram of chargingstation.1.mode
duration         50000    

value                     duration     %
IDLE                      2480.565   5.0 ***
PROC                     47519.435  95.0 ****************************************************************************

